## Parameters

In the next two cells, we import the required Python libraries, provide the access to instances and set up the sets of sorting rules.

In [1]:
import pandas as pd
import os

In [2]:
# Path to problem instances and list their names
instance_path_abc = 'TestInstances/ABC/'
instance_path_suerie = 'TestInstances/Suerie/'
file_names_abc = [file for file in os.listdir(instance_path_abc) if file.endswith('.txt')]
file_names_suerie = [file for file in os.listdir(instance_path_suerie) if file.endswith('.txt')]

# Criteria for sorting rules
cost_crit = ['CUI', 'EC','ES','ESC', 'H', 'HS', 'HSC', 'S','SH','TBO']
period_crit = ['CUP', 'DECP','INCP']
capacity_crit = ['D','DH', 'DHS', 'DS', 'DSH']
key_crit = ['IP', 'PI']

# All sorting rules (75)
sorting_rules = []
for pc in period_crit:
    for ic in cost_crit:
        for k in key_crit:
            rule = pc+'-'+ic+'-'+k
            sorting_rules.append(rule)
for pc in period_crit:
    for dc in capacity_crit:
        rule = dc+'-'+k
        sorting_rules.append(rule)

# Reduced set of soring rules (29)
reduced_set = ['CUP-CUI-IP', 'CUP-DH', 'CUP-DS', 'CUP-ESC-PI', 'CUP-ES-IP', 'CUP-H-IP', 'CUP-SH-PI', 'CUP-TBO-IP',
               'DECP-CUI-IP', 'INCP-CUI-IP', 'INCP-CUI-PI', 'INCP-D', 'INCP-DH', 'INCP-DHS', 'INCP-DS', 'INCP-DSH',
               'INCP-EC-IP', 'INCP-EC-PI', 'INCP-ESC-IP', 'INCP-ESC-PI', 'INCP-ES-PI', 'INCP-H-IP', 'INCP-H-PI',
               'INCP-HSC-IP', 'INCP-HS-PI', 'INCP-SH-IP', 'INCP-SH-PI', 'INCP-S-IP', 'INCP-TBO-IP']

In [3]:
print('Parameters for 2-SCH\n')
print('Path to instance files:\n', instance_path_abc, '\n', instance_path_suerie)
print('\nComplete set of sorting rules:\n', sorting_rules)
print('\nReduced set of sorting rules:\n', reduced_set)

Parameters for 2-SCH

Path to instance files:
 TestInstances/ABC/ 
 TestInstances/Suerie/

Complete set of sorting rules:
 ['CUP-CUI-IP', 'CUP-CUI-PI', 'CUP-EC-IP', 'CUP-EC-PI', 'CUP-ES-IP', 'CUP-ES-PI', 'CUP-ESC-IP', 'CUP-ESC-PI', 'CUP-H-IP', 'CUP-H-PI', 'CUP-HS-IP', 'CUP-HS-PI', 'CUP-HSC-IP', 'CUP-HSC-PI', 'CUP-S-IP', 'CUP-S-PI', 'CUP-SH-IP', 'CUP-SH-PI', 'CUP-TBO-IP', 'CUP-TBO-PI', 'DECP-CUI-IP', 'DECP-CUI-PI', 'DECP-EC-IP', 'DECP-EC-PI', 'DECP-ES-IP', 'DECP-ES-PI', 'DECP-ESC-IP', 'DECP-ESC-PI', 'DECP-H-IP', 'DECP-H-PI', 'DECP-HS-IP', 'DECP-HS-PI', 'DECP-HSC-IP', 'DECP-HSC-PI', 'DECP-S-IP', 'DECP-S-PI', 'DECP-SH-IP', 'DECP-SH-PI', 'DECP-TBO-IP', 'DECP-TBO-PI', 'INCP-CUI-IP', 'INCP-CUI-PI', 'INCP-EC-IP', 'INCP-EC-PI', 'INCP-ES-IP', 'INCP-ES-PI', 'INCP-ESC-IP', 'INCP-ESC-PI', 'INCP-H-IP', 'INCP-H-PI', 'INCP-HS-IP', 'INCP-HS-PI', 'INCP-HSC-IP', 'INCP-HSC-PI', 'INCP-S-IP', 'INCP-S-PI', 'INCP-SH-IP', 'INCP-SH-PI', 'INCP-TBO-IP', 'INCP-TBO-PI', 'D-PI', 'DH-PI', 'DHS-PI', 'DS-PI', 'DSH-PI'

### 2-SCH Main code
In the next cells, we load the Cython Jupyter extension and define classes and methods for 2-SCH in Cython. 

In [4]:
%load_ext cython

In [5]:
%%cython
# cython: boundscheck=False, wraparound=False, initializedcheck=False, nonecheck=False, cdivision=True

DEF MyAbsTol = 1e-12
DEF MyRelTol = 1e-5

import numpy as np
cimport numpy as np
from libc.math cimport fmin, fmax, sqrt, round, ceil
ctypedef np.uint8_t uint8
import random

cdef class DemandList:
    """Class for a list of demand elements.
    It provides a fast way of handling a list of demand elements

    Data:
        length (int): number of elements in the demand list (= length of arrays)
        item_index (c_item_index) (int-ndarray): array of item indices ordered according to the demand list (C memoryview)
        period_index (c_period_index) (int-ndarray): array of period indices ordered according to the demand list (C memoryview)
        amounts (c_amounts) (double-ndarray): array of demand amounts ordered according to the demand list (C memoryview)
        
        (item_index[n], period_index[n], amounts[n]) = n-th demand element

    Methods:
        __init__(length=0): initialize the class and the arrays of the given length.
        load_from_demandMatrix (demand, nr_items, nr_periods): initialize demand list from the demand matrix
        load_from_p_DemandList (demand, p_DemandList): initialize demand list from the python list
        copy (pos1, pos2): return a copy of the demand list between positions pos1 and po2
        swap (pos1, pos2): swap demand elements between positions pos1 and pos2
        get_p_DemandList (): return a python demand list """

    # definition of class variables 
    cdef public np.ndarray item_index, period_index, amounts, cost_upto
    cdef public int length
    cdef int[:] c_item_index, c_period_index
    cdef double[:] c_amounts, c_cost_upto
    
    # constructor
    # set up data structures for a specific length
    def __init__(self, int length = 0):
        self.length = length
        self.item_index = np.zeros(self.length, dtype=int)
        self.period_index = np.zeros(self.length, dtype=int)
        self.amounts = np.zeros(self.length, dtype=float)
        self.c_item_index = self.item_index
        self.c_period_index = self.period_index
        self.c_amounts = self.amounts

    # load data from demand matrix
    cdef void load_from_demandMatrix (self, double[:,:] demand, int nr_items, int nr_periods):
        cdef int i, t
        cdef int pos = 0
        if (nr_items*nr_periods != self.length):             # count nonzero demand entries in the matrix 
            self.length = 0
            for i in range(nr_items):
                for t in range (nr_periods):
                    if demand[i,t] > MyAbsTol:
                        self.length += 1
            self.__init__(self.length)
        for i in range(nr_items):                           # build demand list
            for t in range (nr_periods):
                if demand[i,t]>0:
                    self.c_item_index[pos] = i
                    self.c_period_index[pos] = t
                    self.c_amounts[pos] = demand[i,t]
                    pos += 1
                    
    # load data from a python demand list [(i,t)] and a demand matrix
    cpdef void load_from_p_DemandList (self, double[:,:] demand, list p_DemandList):
        cdef int pos
        if (len(p_DemandList) != self.length):
            self.__init__(len(p_DemandList))
        for pos in range(len(p_DemandList)):
            self.c_item_index[pos] = <int> p_DemandList[pos][0]
            self.c_period_index[pos] = <int> p_DemandList[pos][1]
            self.c_amounts[pos] = <double> demand[p_DemandList[pos][0],p_DemandList[pos][1]]
    
    # create a copy of the demand list
    cpdef DemandList copy (self, int pos1=0, int pos2=0):
        dl = DemandList(self.length)
        if pos2 == 0:
            pos2 = self.length
        dl.c_item_index[pos1:pos2] = self.c_item_index[pos1:pos2]
        dl.c_period_index[pos1:pos2] = self.c_period_index[pos1:pos2]
        dl.c_amounts[pos1:pos2] = self.c_amounts[pos1:pos2]
        return dl
        
    # swap elements on positions pos1 and pos2 in the demand list
    cdef void swap(self,int pos1, int pos2):
        self.c_item_index[pos1], self.c_item_index[pos2] = self.c_item_index[pos2], self.c_item_index[pos1]
        self.c_period_index[pos1], self.c_period_index[pos2] = self.c_period_index[pos2], self.c_period_index[pos1]
        self.c_amounts[pos1], self.c_amounts[pos2] = self.c_amounts[pos2], self.c_amounts[pos1]
    
    # get a python demand list
    def get_p_DemandList(self):
        cdef int pos
        dl = []
        for pos in range(self.length):
            dl.append((self.c_item_index[pos], self.c_period_index[pos]))
        return dl
    
    
cdef class CLSP_data:
    """Class for capacitated lot sizing problems instances.
    Can load data from text files (ABC or Trigeiro instances).
    
    Data:
        instanceName (string): name of the loaded instance
        averageTBO_from_file (string): TBO level read from data file
        
        items (int): number of items
        periods (int): number of periods
        nonzerodemand (int): number of nonzero (> 1e-12) entries in the demand matrix 
        
        demand (numpy.array, dimension: items,periods): demand for each item and period
        capacity (numpy.array, dimension: periods): capacities for each period
        initInventory (numpy.array, dimension: items): initial inventory for each item
        prodCoeff (numpy.array, dimension: items): capacity consumption for each item
        holdingCost (numpy.array, dimension: items): holding cost for each item (per unit, per period)
        setupCost (numpy.array, dimension: items): setup cost for each item
        setupTime (numpy.array, dimension: items): capacity consumption of setup operation for each item
        
        TBO (numpy.array, dimension: items): TBO of each item
        averageDemand (numpy.array, dimension: items): average demand of each item
        
        averagePeriodDemand (double): avergae total demand per period 
        capTightness (double): average capac ity tightness (=averagePeriodDemand/mean per period capacity)
        averageTBO (double): average TBO of all items
        lumpiness (double): percentage of zero entries in the demand matrix
        stdvDemand (double): standard deviation of the demand
        
        overtimeCost (double): overtime cost per unit capacity (set to 10000, not loaded from instance file)
        
        c_... C memoryviews of numpy arrays

    Methods:
        __init__(path, filename, setuptimezero=True): initialize the class and load the data from the given instance file;
                                setup time is set to 0 (setuptimezero=True) or read from the file (setuptimezero=False). """
    
    cdef public str instanceName, averageTBO_from_file
    cdef public int items, periods, nonzerodemand
    cdef public np.ndarray demand, capacity, initInventory, prodCoeff, holdingCost, setupCost, setupTime
    cdef public np.ndarray TBO, averageDemand
    cdef public double averagePeriodDemand, capTightness, averageTBO, lumpiness, stdvDemand
    cdef public double overtimeCost
    cdef double[:,:] c_demand
    cdef double[:] c_capacity
    cdef double[:] c_initInventory
    cdef double[:] c_prodCoeff
    cdef double[:] c_holdingCost
    cdef double[:] c_setupCost
    cdef double[:] c_setupTime
    cdef double[:] c_TBO
    cdef double[:] c_averageDemand
        
    # initialize CLPS_data class - reads data from instnace file
    def __init__(self, path, filename, setuptimezero = True):
        demand = []; capacity = []; initinv = []; holdc = []; prodcoeff = []; setupc = []; setupt = [] # initialize lists
        with open(path + filename,"r") as iFile:               # read data from file
            self.instanceName = filename    # name of instance
            vals0 = iFile.readline().strip().split()
            if len(vals0) > 3:
                self.averageTBO_from_file = vals0[3]
            iFile.readline()
            vals = iFile.readline().strip().split()
            self.items = int(vals[0])
            self.periods = int(vals[1])
            iFile.readline()                        
            values = iFile.readline().strip().split()                   # read capacity
            for val in values:
                capacity.append(float(val))
            self.capacity = np.array(capacity, dtype=float)
            self.c_capacity = self.capacity
            iFile.readline()
            values = iFile.readline().strip().split()                   # read initial inventory
            for val in values:
                initinv.append(float(val))
            iFile.readline()
            values = iFile.readline().strip().split()                   # read holding cost
            for val in values:
                holdc.append(float(val))
            iFile.readline(); iFile.readline(); iFile.readline()        # skip end inventory
            for i in range(0, self.items):                              # read demand
                values = iFile.readline().strip().split()
                demand_i = []
                for val in values:
                    demand_i.append(float(val))
                demand.append(demand_i.copy())
            iFile.readline()
            for i in range(0, self.items):                             # read prod coefficients
                values = iFile.readline().strip().split()
                prodcoeff.append(float(values[2]))
            self.prodCoeff = np.array(prodcoeff, dtype=float)
            self.c_prodCoeff = self.prodCoeff
            self.holdingCost = np.array(holdc, dtype=float)
            self.holdingCost = self.holdingCost/self.prodCoeff
            self.c_holdingCost = self.holdingCost
            for i in range(0, self.items):
                for p in range(0, self.periods):
                    demand[i][p] = demand[i][p]*prodcoeff[i]            # convert demand to capacity units
            self.demand = np.array(demand, dtype=float, order="C")
            self.c_demand = self.demand
            for i in range(0, self.items):
                initinv[i] = initinv[i]*prodcoeff[i]                    # convert initial inventory to capacity units
            self.initInventory = np.array(initinv, dtype=float)
            self.c_initInventory = self.initInventory
            iFile.readline()
            for i in range(0, self.items):                              # read setup costs
                setupc.append(float(iFile.readline().strip()))
            self.setupCost = np.array(setupc, dtype=float)
            self.c_setupCost = self.setupCost
            iFile.readline()
            for i in range(0, self.items):                             # read setup times
                values = iFile.readline().strip().split()
                if setuptimezero:                                      # default is setup time = 0, otherwise read from file
                    setupt.append(0.0)
                else:
                    setupt.append(float(values[2]))
            self.setupTime = np.array(setupt, dtype=float)
            self.c_setupTime = self.setupTime
            self.overtimeCost = 10000
            self.nonzerodemand = 0                                     # count non-zero demand elements without first period
            for i in range(self.items):
                for t in range(1,self.periods):
                    if (self.c_demand[i][t] > MyAbsTol):
                        self.nonzerodemand += 1
            self.averageDemand = np.zeros(self.items, dtype=float)     # compute average demand of each item
            self.c_averageDemand = self.averageDemand
            for i in range(self.items):
                for t in range(self.periods):
                    self.c_averageDemand[i] += self.c_demand[i][t]
                self.c_averageDemand[i] /= self.periods
            self.averagePeriodDemand = 0.0                             # compute average period demand (in capacity units)
            for i in range(self.items):
                for t in range(self.periods):
                    self.averagePeriodDemand += self.c_demand[i][t]
            self.averagePeriodDemand /= self.periods
            self.TBO = np.zeros(self.items, dtype=float)               # compute TBO of each item and average TBO
            self.c_TBO = self.TBO
            self.averageTBO = self.TBO.mean()
            averageSetupCap = 0
            for i in range(self.items):
                self.c_TBO[i] = sqrt((2*self.c_setupCost[i])/(self.c_holdingCost[i]*self.averageDemand[i]))
                averageSetupCap += self.c_setupTime[i] / self.c_TBO[i]
            self.capTightness = (self.averagePeriodDemand + averageSetupCap) / self.capacity.mean()     # compute capacity tightness WITH setup times
            self.lumpiness = 0
            for i in range(self.items):
                for t in range(self.periods):
                    if self.c_demand[i][t] <= MyAbsTol:
                        self.lumpiness += 1
            self.lumpiness /= (self.items*self.periods)     # compute lumpiness
            self.stdvDemand = np.std(self.demand)           # compute standard deviation of demand

            
cdef class CLSP(CLSP_data):
    """General CLSP class (derived from CLSP_data).
    It loads data for instance file and generates solution data fields.
    
    Data and variables:
      instance related data (from CLSP_data):
        instanceName (string): name of the loaded instance
        items (int): number of items
        periods (int): number of periods
        demand (numpy.array, dimension: items,periods): demand for each item and period
        capacity (numpy.array, dimension: periods): capacities for each period
        initInventory (numpy.array, dimension: items): initial inventory for each item
        prodCoeff (numpy.array, dimension: items): capacity consumption for each item
        holdingCost (numpy.array, dimension: items): holding cost for each item (per unit, per period)
        setupCost (numpy.array, dimension: items): setup cost for each item
        setupTime (numpy.array, dimension: items): capacity consumption of setup operation for each item
        overtimeCost (double): overtime cost per unit capacity (set to 10000, not loaded from instance file)

      production plan related data:
        lotsizes (numpy.array (double), dimension: items,periods): lot sizes (production plan)
        setup (numpy.array (bool), dimension: items,periods): setup decisions
        inventory (numpy.array (double), dimension: items,periods): inventory levels at the end of each period
        avCap (numpy.array (double), dimension: periods): available capacity in production plan
        totalHoldingCost (double): total holding cost of production plan
        totalSetupCost (double): total setup cost of production plan
        totalOvertimeCost (double): total overtime cost of production plan
        totalCost (double): total cost of production plan
        
        ..._store: storage containers for the variables for storing solutions (first index gives the number of the storage container)
        
        c_... C memoryviews of numpy arrays and Python variables
      
      2-SCH handling variables:
        cost_amounts (numpy.array, dimensions: periods): lot sizes (incremental) for item i reported after checking EXT and NEW options
        postponementDL (DemandList): postponement demand list
        deactivatePostponement (bool): postponement routine must be deactivated in case I when we use inventory
        postpone (bool): demand element should be put in the postponement demand list (=1) or not (=0)
        DL (bool): work with main demand list (=0) or postponement demand list (=1)
        usePostDL (bool): postponement demand list must be checked (=1) or not (=0)
        notAllNow (bool): indicates whether it is possible to add the whole demand element to its period (=0) or not (=1)
        keepInPostDL (bool): demand element from postponement demand list is postponed again (=1) or not (=0)
        nextPos (int): the position after the last element in the postponement demand list
      
      2-SCH versions control:
        shiftProduction (bool): shift of production routine is enabled (=1) or not (=0)
        postponemnt (bool): postponement routine is enabled (=1) or not (=0)
        threshold (double): threshold value
      
    Methods:
        __init__(path, filename, storage): initialize the class and load the data from the given instance file. The demand of the first period is planned.
        __str__(): convert solution into string for output.
        resetSolution(): set the production plan to the initial (actual plan of the first period demand).
        storeSolution(container): store current solution in container (container=0 is reserved for resetSolution-function)
        retrieveSolution(container): retrieve solution from container
        update_totalHoldingCost(): update the total holding cost and return it
        update_totalSetupCost(): update the total setup cost and return it
        update_totalOvertimeCost(): update the total overtime cost and return it
        update_totalCost(): update the total cost and return it
        reset_postponementDL(): empty postponement demand list
        try_addLot2Period(i, t, tt, amount, maxamount, dummyCost): try to add amount units of item i, used to satisy demand in period t, to the production in period tt
        addLot2Period(i, t, tt, amount): actually add amount units of item i, used to satisy demand in period t, to the production in period tt
        _useInv ( i, t, amount): case I of 2-SCH - use existing inventory to satisfy (partially) demand of amount units of item i in t
        try_EXTandNEW(i, t, amount): options EXT and NEW - compute incremental cost of extending existing lots (EXT) and/or creating a new lot (NEW) to satisfy demand of amount units of item i in period t
        add_EXTorNEW(i, t): execute options EXT / NEW for demand of item i in period t
        postponeDemand(i, t, amount, firstPostponement): put demand element of amount units of item i in period t to the postponement demand list
        _useOvertime(i, t, amount): case IV of 2-SCH - use overtime to satisfy demand of amount units of item i in t
        addDemand(i, t, amount, firstPostponement): add demand of amount units of item i in period t to the production plan, firstPostponement indicates whether this demand element is already in postponement list or not yet
        sortItems(criterion): demand elements sorted according to an item-based criterion
        sortPeriods(criterion): demand elements sorted according to a period-based criterion
        generateDLOnline(criterionPeriod, criterionItem, choice): generate demand list with period-based and cost-based criteria, choice indicates which criterion is the primary key
        generateDLdemandOnline(criterionPeriod, criterionDemand): generate demand list with period-based and capacity-based criteria
        generateOnline(mode): generate demand list for random or realistic scenario
        accumulativeSort(dummyDL, mode, criterionPeriod, criterionDemand, criterionItem, choice): sort demand list (with incomplete demand matrix) for accumulative scenario
        planDemandList(dl, resetSol, pos1, pos2, resetPostDL): create a production plan using demand list dl for demand elements between pos1 and pos2, resetSol indicates whether should start from empty production plan, resetPostDL - whether postponement demand list must be emptied.
        """
    
    # definition of class variables
    cdef public np.ndarray lotsizes, setup, inventory, avCap, cost_amounts
    cdef public double totalHoldingCost, totalSetupCost, totalOvertimeCost, totalCost
    cdef double[:,:] c_lotsizes
    cdef uint8[:,:] c_setup
    cdef double[:,:] c_inventory
    cdef double[:] c_avCap
    cdef double[:] c_cost_amounts
    
    # storage containers for the results
    cdef np.ndarray lotsizes_store, setup_store, inventory_store, avCap_store
    cdef np.ndarray totalHoldingCost_store, totalSetupCost_store, totalOvertimeCost_store, totalCost_store
    cdef double[:,:,:] c_lotsizes_store
    cdef uint8[:,:,:] c_setup_store
    cdef double[:,:,:] c_inventory_store
    cdef double[:,:] c_avCap_store
    cdef double[:] c_totalHoldingCost_store
    cdef double[:] c_totalSetupCost_store
    cdef double[:] c_totalOvertimeCost_store
    cdef double[:] c_totalCost_store
    
    #parameters for 2-SCH
    cdef uint8 shiftProduction, postponement
    cdef double threshold
    
    cdef uint8 deactivatePostponement, postpone, DL, usePostDL, notAllNow, keepInPostDL
    cdef int nextPos
    cdef DemandList postponementDL
    
    # constructor - calls CLSP_data.__init__, storage gives the number of initialized storage containers
    def __init__(self, str path, str filename, uint8 setuptimezero = False, int storage = 1,
                 uint8 shiftProduction = True, uint8 postponement = True, double threshold = 0.5):
        # define 2-SCH version
        self.shiftProduction = shiftProduction
        self.postponement = postponement
        self.threshold = threshold
        
        # instance and production plan related data
        CLSP_data.__init__ (self, path, filename, setuptimezero)
        self.lotsizes = np.zeros((self.items,self.periods), dtype=float, order="C")
        self.c_lotsizes = self.lotsizes
        self.setup = np.zeros((self.items,self.periods),dtype=np.uint8,order="C")
        self.c_setup = self.setup
        self.inventory = np.zeros((self.items,self.periods), dtype=float, order="C")
        self.c_inventory = self.inventory
        self.avCap = np.zeros(self.periods,dtype=float)
        self.c_avCap = self.avCap
        self.c_avCap[...] = self.c_capacity
        
        # extra storage data
        self.lotsizes_store = np.zeros((storage+1,self.items,self.periods), dtype=float, order="C")
        self.c_lotsizes_store = self.lotsizes_store
        self.setup_store = np.zeros((storage+1,self.items,self.periods),dtype=np.uint8,order="C")
        self.c_setup_store = self.setup_store
        self.inventory_store = np.zeros((storage+1,self.items,self.periods), dtype=float, order="C")
        self.c_inventory_store = self.inventory_store
        self.avCap_store = np.zeros((storage+1,self.periods),dtype=float, order="C")
        self.c_avCap_store = self.avCap_store
        self.totalHoldingCost_store = np.zeros(storage+1,dtype=float)
        self.c_totalHoldingCost_store = self.totalHoldingCost_store
        self.totalSetupCost_store = np.zeros(storage+1,dtype=float)
        self.c_totalSetupCost_store = self.totalSetupCost_store
        self.totalOvertimeCost_store = np.zeros(storage+1,dtype=float)
        self.c_totalOvertimeCost_store = self.totalOvertimeCost_store
        self.totalCost_store = np.zeros(storage+1,dtype=float)
        self.c_totalCost_store = self.totalCost_store
        
        # initialize handling variables
        self.cost_amounts = np.zeros(self.periods, dtype=float)
        self.c_cost_amounts = self.cost_amounts
        self.postpone = 0
        self.usePostDL = 1
        self.DL = 0
        self.nextPos = 0
        self.keepInPostDL = False
        self.notAllNow = False
        self.deactivatePostponement = False
        
        # initialize postponement demand list
        self.postponementDL = DemandList(self.nonzerodemand)
        
        # plan demand in the first period and determine the cost of production plan
        self.totalHoldingCost = 0.0
        self.totalSetupCost = 0.0
        for i in range(self.items):
            if (self.c_demand[i][0] > MyAbsTol):
                self.c_lotsizes[i][0] = self.c_demand[i][0]
                self.c_avCap[0] -= self.c_demand[i][0] + self.c_setupTime[i]
                self.c_setup[i][0] = True
                self.totalSetupCost += self.c_setupCost[i]
        if self.c_avCap[0] < -MyAbsTol:
            self.totalOvertimeCost = self.overtimeCost * (-self.c_avCap[0])
        else:
            self.totalOvertimeCost = 0.0
        self.totalCost = self.totalSetupCost + self.totalOvertimeCost
        self.storeSolution(0)
    
    # create a string with production plan data 
    def __str__(self):
        ostring = self.instanceName + "\n"
        ostring += "Cost: " + str(self.totalCost) + "(S: " + str(self.totalSetupCost) + ", H: " + str(self.totalHoldingCost) + ", O: " + str(self.totalOvertimeCost) + ")\n"
        ostring += "Production plan: \n" + str(self.lotsizes) + "\n"
        ostring += "Inventory: \n" + str(self.inventory) + "\n"
        ostring += "Available capacity: \n" + str(self.avCap) + "\n"
        return ostring
    
    # reset production plan to initial
    cpdef void resetSolution(self):
        self.retrieveSolution(0)
    
    # store solution in container (container=0 is reserved)
    cpdef void storeSolution(self, int container = 1):
        self.c_lotsizes_store[container][...] = self.c_lotsizes
        self.c_setup_store[container][...] = self.c_setup
        self.c_inventory_store[container][...] = self.c_inventory
        self.c_avCap_store[container][...] = self.c_avCap
        self.c_totalHoldingCost_store[container] = self.totalHoldingCost
        self.c_totalSetupCost_store[container] = self.totalSetupCost
        self.c_totalOvertimeCost_store[container] = self.totalOvertimeCost
        self.c_totalCost_store[container] = self.totalCost
    
    # retrieve solution in container (container=0 is reserved)
    cpdef void retrieveSolution(self, int container = 1):
        self.c_lotsizes[...] = self.c_lotsizes_store[container]
        self.c_setup[...] = self.c_setup_store[container]
        self.c_inventory[...] = self.c_inventory_store[container]
        self.c_avCap[...] = self.c_avCap_store[container]
        self.totalHoldingCost = self.c_totalHoldingCost_store[container]
        self.totalSetupCost = self.c_totalSetupCost_store[container]
        self.totalOvertimeCost = self.c_totalOvertimeCost_store[container]
        self.totalCost = self.c_totalCost_store[container]

    # recalculate the holding cost
    cdef double update_totalHoldingCost(self):
        self.totalHoldingCost = 0
        cdef int i,p
        cdef double sumInvP
        for i in range(self.items):
            sumInvP = 0
            for p in range(self.periods):
                sumInvP += self.c_inventory[i][p]
            self.totalHoldingCost += sumInvP * self.c_holdingCost[i]
        return self.totalHoldingCost

    # recalculate setup cost
    cdef double update_totalSetupCost(self):
        self.totalSetupCost = 0
        cdef int i,p
        cdef double sumSetupP
        for i in range(self.items):
            sumSetupP = 0
            for p in range(self.periods):
                sumSetupP += self.c_setup[i][p]
            self.totalSetupCost += sumSetupP * self.c_setupCost[i]
        return self.totalSetupCost

    # recalculate overtime cost
    cdef double update_totalOvertimeCost(self):
        self.totalOvertimeCost = 0
        cdef int p
        for p in range(self.periods):
            if self.c_avCap[p] < -MyAbsTol:
                self.totalOvertimeCost -= self.c_avCap[p]
        self.totalOvertimeCost *= self.overtimeCost
        return self.totalOvertimeCost

    # recalculate total cost
    cpdef double update_totalCost(self):
        self.totalCost = self.update_totalHoldingCost() + self.update_totalSetupCost() + self.update_totalOvertimeCost()
        return self.totalCost
    
    # reset postponement demand list
    cpdef void reset_postponementDL(self):
        self.postponementDL = DemandList(self.nonzerodemand)
    
    # try to add a lot (demand for period t) to a specific period (tt)
    # returns: cost (incremental); maxamount (maximum amount that can be added) is returned via pointer
    cdef double try_addLot2Period(self, int i, int t, int tt, double amount, double* maxamount, uint8 dummyCost=False):
        
        cdef uint8 activeShift = False # control if shift is enabled
        
        if amount <= MyAbsTol:
            maxamount[0] = 0
            return 0
        
        cdef double scost = 0     # setup cost (incremental)
        
        if self.c_setup[i][tt] == False or self.shiftProduction == True:
            activeShift = True
        
        # compute the amount that can be added 
        if self.c_setup[i][tt]:      
            maxamount[0] = fmin((self.c_avCap[tt]),amount)
        else:                      
            maxamount[0] = fmin((self.c_avCap[tt]-self.c_setupTime[i]),amount)
            scost = self.c_setupCost[i]    # if there was no setup, there is additional setup cost
        
        if maxamount[0] <= MyAbsTol:       # no production can be added to period tt
            maxamount[0] = 0
            return -1
        
        #local variables to handle shifts
        cdef double cost_sav_inv = 0
        cdef double add_cap_inv = 0
        cdef int tt_back
        cdef double maxfrominv
        cdef double cost_sav_future = 0
        cdef double avcap_future = 0
        cdef int tt_future = tt + 1
        
        if activeShift:
            # right shift
            if tt>0:
                if self.c_inventory[i][tt-1] > MyAbsTol:
                    maxfrominv = fmin(self.c_inventory[i][tt-1],self.c_avCap[tt]-maxamount[0])
                    add_cap_inv = maxfrominv   
                    if maxfrominv > MyAbsTol:
                        tt_back = tt-1
                        while maxfrominv > MyAbsTol:
                            while self.c_setup[i][tt_back] == False:
                                cost_sav_inv += self.c_holdingCost[i] * maxfrominv
                                tt_back -= 1
                            cost_sav_inv += self.c_holdingCost[i] * maxfrominv
                            if (maxfrominv - self.c_lotsizes[i][tt_back])>= -MyAbsTol:
                                maxfrominv -= self.c_lotsizes[i][tt_back]
                                cost_sav_inv += self.c_setupCost[i]
                            else:
                                break
                            tt_back -= 1     
            
            # left shift
            avcap_future = self.c_avCap[tt]-maxamount[0]-add_cap_inv 
            while (avcap_future > MyAbsTol and tt_future < self.periods):
                if self.c_setup[i][tt_future]==True:
                    amount_future = self.c_lotsizes[i][tt_future]
                    if dummyCost:
                        amount_future += self.c_cost_amounts[tt_future]
                    if (avcap_future >= amount_future-MyAbsTol and 
                        amount_future*self.c_holdingCost[i]*(tt_future-tt) < self.c_setupCost[i]):
                        avcap_future -= amount_future
                        cost_sav_future += self.c_setupCost[i] - amount_future*self.c_holdingCost[i]*(tt_future-tt)
                    else:
                        break
                tt_future += 1
        
        return scost + maxamount[0]*self.c_holdingCost[i]*(t-tt) - cost_sav_inv - cost_sav_future
    
    # adds the amount (demand for period t) to production in period tt
    # returns total cost of current plan
    cdef double addLot2Period(self, int i, int t, int tt, double amount):
        cdef uint8 activeShift = False
        
        if amount <= MyAbsTol:
            return self.totalCost
        
        self.c_lotsizes[i][tt] += amount
        
        cdef double avCapprv = self.c_avCap[tt]
        self.c_avCap[tt] -= amount
        
        cdef int p
        for p in range(tt,t):
            self.c_inventory[i][p] += amount
        self.totalHoldingCost += (t-tt) * self.c_holdingCost[i] * amount
        if self.c_setup[i][tt] == False or self.shiftProduction == True:
            activeShift = True
        if self.c_setup[i][tt] == False:
            self.c_setup[i][tt] = True
            self.c_avCap[tt] -= self.c_setupTime[i]
            self.totalSetupCost += self.c_setupCost[i]
            
        if self.c_avCap[tt] < -MyAbsTol:
            self.totalOvertimeCost += (-self.c_avCap[tt])*self.overtimeCost - (fmax(-avCapprv,0)*self.overtimeCost)
        self.totalCost = self.totalHoldingCost + self.totalSetupCost + self.totalOvertimeCost

        cdef double maxfrominv
        cdef int tt_back

        cdef int tt_future = tt + 1
        cdef int ttt
        
        if activeShift:
            # right shift
            if tt>0:
                if self.c_inventory[i][tt-1] > MyAbsTol:
                    maxfrominv = fmin(self.c_inventory[i][tt-1],self.c_avCap[tt])
                    if maxfrominv > MyAbsTol:
                        self.c_lotsizes[i][tt] += maxfrominv
                        self.c_avCap[tt] -= maxfrominv
                        tt_back = tt-1
                        while maxfrominv > MyAbsTol:
                            while self.c_setup[i][tt_back] == False:
                                self.c_inventory[i][tt_back] -= maxfrominv
                                self.totalHoldingCost -= self.c_holdingCost[i] * maxfrominv
                                tt_back -= 1
                            self.c_inventory[i][tt_back] -= maxfrominv
                            self.totalHoldingCost -= self.c_holdingCost[i] * maxfrominv
                            if (maxfrominv - self.c_lotsizes[i][tt_back])>= -MyAbsTol:
                                maxfrominv -= self.c_lotsizes[i][tt_back]
                                self.c_setup[i][tt_back] = False
                                self.totalSetupCost -= self.c_setupCost[i]
                                self.c_avCap[tt_back] += self.c_lotsizes[i][tt_back] + self.c_setupTime[i]  
                                self.c_lotsizes[i][tt_back] = 0
                            else:
                                self.c_avCap[tt_back] += maxfrominv
                                self.c_lotsizes[i][tt_back] -= maxfrominv
                                break
                            tt_back -= 1
            # left shift
            while (self.c_avCap[tt] > MyAbsTol and tt_future < self.periods):
                if self.c_setup[i][tt_future]==True:
                    if (self.c_avCap[tt] >= self.c_lotsizes[i][tt_future]-MyAbsTol and
                        self.c_lotsizes[i][tt_future]*self.c_holdingCost[i]*(tt_future-tt) < self.c_setupCost[i]):
                        self.c_avCap[tt] -= self.c_lotsizes[i][tt_future]
                        self.c_lotsizes[i][tt] += self.c_lotsizes[i][tt_future]
                        self.c_setup[i][tt_future] = False
                        self.totalSetupCost -= self.c_setupCost[i]
                        for ttt in range(tt,tt_future):
                            self.c_inventory[i][ttt] += self.c_lotsizes[i][tt_future]
                        self.totalHoldingCost += self.c_lotsizes[i][tt_future]*self.c_holdingCost[i]*(tt_future-tt)
                        self.c_avCap[tt_future] += self.c_lotsizes[i][tt_future] + self.c_setupTime[i]   
                        self.c_lotsizes[i][tt_future] = 0.0
                    else:
                        break
                tt_future += 1
        
        self.update_totalCost()

        return self.totalCost
    
    # case I of 2-SCH: use existing inventory to satisfy (partially) demand
    # returns remaining amount to be scheduled
    cdef double _useInv (self, int i, int t, double amount):
        cdef int p
        cdef double avCap = 0
        cdef double possible_amount = 0
        cdef double remaining_amount = 0
        if self.c_inventory[i][t] > MyAbsTol:     # if there is positive inventory
            for p in range(t+1,self.periods):
                avCap += fmax(self.c_avCap[p],0)
            if avCap > MyAbsTol:                  # and if there is available capacity for production after period t
                possible_amount = fmin(fmin(self.c_inventory[i][t], amount),avCap) # compute what is the amount that can be satisfied from inventory: = d^new
                remaining_amount = amount - possible_amount # compute the remaining amount: =d^rem
                self.c_inventory[i][t] -= possible_amount
                self.totalHoldingCost -= self.c_holdingCost[i] * possible_amount
                self.totalCost -= self.c_holdingCost[i] * possible_amount
                self.deactivatePostponement = True
                self.addDemand(i, t + 1, possible_amount, True) # add the "new demand element" in the next period immediately
                return remaining_amount
        return amount
            
    # options EXT and NEW: compute backwards cost of extending existing lots and/or creating new lots
    # returns cost (incremental) of the cheapest option
    cdef double try_EXTandNEW(self, int i, int t, double amount):
        cdef uint8 NoSetupPeriodFound = False
        cdef double costEXT = 0
        cdef double costNEW = 1e+24, thiscost = 1e+24
        cdef int tt = t-1
        cdef int ttt = t
        cdef double amount2 = 0
        cdef double possible_amount_rem = amount
        cdef double possible_amount_3 = 0
        self.c_cost_amounts[...] = 0
        self.postpone = 0

        while possible_amount_rem > MyAbsTol and tt >= 0:           
            costEXT += possible_amount_rem * self.c_holdingCost[i]
            amount2 = self.c_avCap[tt]
            if self.c_setup[i][tt] and amount2 > MyAbsTol:
                possible_amount_rem -= amount2
                self.c_cost_amounts[tt] = min(possible_amount_rem+amount2, amount2)
            if not self.c_setup[i][tt]:
                amount2 -= self.c_setupTime[i]       
                if amount2 >= possible_amount_rem:
                    if not NoSetupPeriodFound:
                        thiscost = self.try_addLot2Period(i, t, tt, possible_amount_rem, &possible_amount_3, dummyCost=False)
                        if possible_amount_3 >= possible_amount_rem:
                            if thiscost < costNEW:
                                costNEW = thiscost
                                self.c_cost_amounts[tt] = possible_amount_rem
                                ttt = tt
                                NoSetupPeriodFound = True
            tt -= 1
            if costEXT > costNEW:
                break
        
        if possible_amount_rem > MyAbsTol:
            costEXT = 1e+24
        
        if self.postponement and (costEXT != 1e+24):
            if (abs(costEXT-costNEW) <= self.threshold * costEXT) and (self.DL == 0) and not self.deactivatePostponement:
                self.postpone = 1
        
        if costEXT <= costNEW:
            self.c_cost_amounts[ttt] = 0
            return costEXT
        else:
            self.c_cost_amounts[:ttt] = 0
            return costNEW
            
    # execute the cheapest option from EXT and NEW
    cdef double add_EXTorNEW(self, int i, int t):
        cdef int tt = t-1
        
        while tt >= 0:
            if tt == 0:
                return self.addLot2Period(i, t, tt, self.c_cost_amounts[tt])
            self.addLot2Period(i, t, tt, self.c_cost_amounts[tt])
            tt -= 1
    
    # put demand element to the postponement demand list
    cdef double postponeDemand(self, int i, int t, double amount, firstPostponement=True):
        cdef int pos, period
        if firstPostponement:
            self.postponementDL.c_item_index[self.nextPos] = i
            self.postponementDL.c_period_index[self.nextPos] = t
            self.postponementDL.c_amounts[self.nextPos] = amount
            self.nextPos += 1
        else:
            for pos in range(0, self.nextPos):
                if i == self.postponementDL.c_item_index[pos] and t == self.postponementDL.c_period_index[pos]:
                    self.postponementDL.c_amounts[pos] = amount
            self.keepInPostDL = True
        self.usePostDL = 0
        return self.totalCost
    
    # case IV of 2-SCH: use overtime
    # returns total cost of current plan
    cdef double _useOvertime(self, int i, int t, double amount):
        cdef int tt = t-1
        cdef int t_inv = tt
        cdef double amount2

        while tt >= 0: # fill available capacity with production in earlier periods as much as possible
            if (self.c_avCap[tt] - (self.c_setupTime[i]*(1-self.c_setup[i][tt]))) > MyAbsTol:
                amount2 = min(amount, (self.c_avCap[tt]- (self.c_setupTime[i]*(1-self.c_setup[i][tt]))))
                if amount2 > MyAbsTol:
                    if self.c_setup[i][tt] == False:
                        self.c_setup[i][tt] = True
                        self.totalSetupCost += self.c_setupCost[i]
                        self.c_avCap[tt] -= self.c_setupTime[i] 
                    self.c_lotsizes[i][tt] += amount2
                    self.c_avCap[tt] -= amount2 
                    for t_inv in range(tt, t):
                        self.c_inventory[i][t_inv] += amount2
                        self.totalHoldingCost += self.c_holdingCost[i]*amount2*(t-t_inv)
                    amount -= amount2
            tt -= 1
        if amount > MyAbsTol: # put the rest as overtime in period t
            if self.c_setup[i][t] == False:
                self.c_setup[i][t] = True
                self.totalSetupCost += self.c_setupCost[i]
                self.c_avCap[t] -= self.c_setupTime[i]   
            self.c_lotsizes[i][t] += amount
            self.c_avCap[t] -= amount
            self.totalOvertimeCost += self.overtimeCost*max(0, -self.c_avCap[t])
        
        self.update_totalCost()
        
        return self.totalCost
    
    # extend partial production plan: manages all cases of 2-SCH for a demand element
    # returns total cost
    cdef double addDemand(self, int i, int t, double amount, uint8 activeUseInv=True, uint8 firstPostponement=True):
        cdef double possible_amount = 0
        cdef double remaining_amount = 0
        cdef double cost = 0
        cdef double other_cost = 0
                
        self.notAllNow = False

        if amount <= MyAbsTol:          # check if amount is positive, otherwise no change of the production plan
            return self.totalCost
        
        # case I: Use inventory
        if (activeUseInv and (t < self.periods-1)): 
            remaining_amount = self._useInv(i,t,amount)
            if remaining_amount <= MyAbsTol:
                return self.totalCost
            self.notAllNow = False
            amount = remaining_amount
        
        # if we are in first period (t=0), we have to add the amount here
        if t == 0:                  
            return self.addLot2Period(i, t, t, amount)
        
        cost = self.try_addLot2Period(i,t,t,amount,&possible_amount)    # compute option NOW
        remaining_amount = amount-possible_amount
        if possible_amount <= MyAbsTol:
            self.notAllNow = True
        
        # case II: enough capacity, everything can be added to t
        if remaining_amount <= MyAbsTol:  # if everything can be added but with additional cost (setup)
            other_cost = self.try_EXTandNEW(i, t, amount)
            if (cost <= other_cost) and not self.notAllNow: #option NOW - cheaper to add everything to t (either create a new lot or extend the existing lot)
                return self.addLot2Period(i, t, t, amount)
            elif (self.DL == 0) and (self.postpone == 1): # POSTPONEMENT routine - cheaper to create/extend previous lot/s but the difference between both is too small, so postpone 
                return self.postponeDemand(i, t, amount, firstPostponement)
            else:
                return self.add_EXTorNEW(i, t) # execute option EXT or NEW
        
        # case III: not enough capacity, only part or nothing can be added to t
        
        # if there are no additional cost, but not the whole amount can be added
        if cost <= 0 and not self.notAllNow:           
            self.addLot2Period(i,t,t,possible_amount)   # add what is possible
            other_cost = self.try_EXTandNEW(i, t, remaining_amount) # try options EXT and NEW for the remaining amount (translates to options NOW+EXT and NOW+NEW options in case III)
            if other_cost == 1e+24:                     # if options EXT and NEW are infeasible (=1e+24)
                return self._useOvertime(i, t, remaining_amount) # case IV: use overtime
            if (self.DL == 0) and (self.postpone == 1): # POSTPONEMENT routine - options EXT and NEW are feasible but too small difference, so postpone
                    return self.postponeDemand(i, t, remaining_amount, firstPostponement)
            else:                                       
                return self.add_EXTorNEW(i, t) # execute option EXT or NEW
        
        # if there is additional cost and not everything can be added to t
        else:                                           
            other_cost2 = self.try_EXTandNEW(i, t, remaining_amount) # try options EXT and NEW for the remaining amount (translates to options NOW+EXT and NOW+NEW options in case III)
            other_cost = self.try_EXTandNEW(i, t, amount)            # try options EXT and NEW for the whole amount
            if (other_cost < other_cost2+cost) and (other_cost2 != 1e+24): # EXT or NEW is cheaper
                other_cost = self.try_EXTandNEW(i, t, amount)
            if other_cost2 == 1e+24:                    # if options EXT and NEW are infeasible (=1e+24)
                return self._useOvertime(i, t, amount)  # case IV: use overtime
            if (cost+other_cost2 <= other_cost) and not self.notAllNow: # NOW+EXT or NOW+NEW is cheaper
                self.addLot2Period(i,t,t,possible_amount)   # add what is possible
                other_cost2 = self.try_EXTandNEW(i, t, remaining_amount)
                if (self.DL == 0) and (self.postpone == 1): # POSTPONEMENT routine - options EXT and NEW are feasible but too small difference, so postpone
                    return self.postponeDemand(i, t, remaining_amount, firstPostponement)
                else:
                    return self.add_EXTorNEW(i, t)          # execute option EXT or NEW
            elif (self.DL == 0) and (self.postpone == 1):
                return self.postponeDemand(i, t, amount, firstPostponement)
            else:
                return self.add_EXTorNEW(i, t)

            
    # sort items according to some criteria
    # returns a sorted list of item indices
    def sortItems(self, str criterion, dummyDemand='Not provided'):
        if dummyDemand == 'Not provided':
            dummyDemand = self.c_demand
        itemlist = []
        for it in range(0, self.items):
            itemlist.append(it)
        if criterion == "SH":
            itemlist.sort(key=lambda x: (self.setupCost[x]/self.holdingCost[x]),reverse=True)
        elif criterion == "HS":
            itemlist.sort(key=lambda x: (self.holdingCost[x]/self.setupCost[x]), reverse=True)
        elif criterion == "TBO":
            itemlist.sort(key=lambda x: (self.TBO[x]), reverse=True)
        elif criterion == "HSC":
            itemlist.sort(key=lambda x: (self.holdingCost[x]/(self.setupCost[x]*self.averageDemand[x])), reverse=True)
        elif criterion == "EC":
            itemlist.sort(key=lambda x: ((self.setupCost[x]/self.TBO[x])+(self.TBO[x]*self.holdingCost[x]*self.averageDemand[x]/2)), reverse=True)
        elif criterion == "ES":
            itemlist.sort(key=lambda x: ((self.TBO[x]-1)*self.setupCost[x]-(self.TBO[x]*(self.TBO[x]-1)*self.holdingCost[x]*self.averageDemand[x]/2)), reverse=True)
        elif criterion == "ESC":
            itemlist.sort(key=lambda x: ((self.TBO[x]-1)*self.setupCost[x]-(self.TBO[x]*(self.TBO[x]-1)*self.holdingCost[x]*self.averageDemand[x]/2))/self.averageDemand[x], reverse=True)
        elif criterion == "CUI":
            itemlist.sort(key=lambda x: (dummyDemand[x, :]).sum(), reverse=True)
        elif criterion == "S":
            itemlist.sort(key=lambda x: self.setupCost[x], reverse=True)
        elif criterion == "H":
            itemlist.sort(key=lambda x: self.holdingCost[x], reverse=True)
        elif criterion == "RAND":
            random.shuffle(itemlist)
        else:
            print("Invalid Criterion!")
        return itemlist

    # sort items according to some criteria
    # returns a sorted list of period indices
    def sortPeriods(self, str criterion, dummyDemand='Not provided'):
        if dummyDemand == 'Not provided':
            dummyDemand = self.c_demand
        periodlist = []
        for p in range(1, self.periods):
            periodlist.append(p)
        if criterion == "INCP":
            return periodlist
        elif criterion == "DECP":
            periodlist.sort(key=lambda x: x, reverse=True)
        elif criterion == "CUP":
            periodlist.sort(key=lambda x: (dummyDemand[:, x]).sum(), reverse=True)
        else:
            print("Invalid Criterion!")
        return periodlist

    # generate demand list with specific sorting critieria (only for cost-based criteria)
    # returns demand list
    def generateDLOnline(self, str criterionPeriod, str criterionItem, str choice, dummyDemand='Not provided'):
        if dummyDemand == 'Not provided':
            dummyDemand = self.c_demand
        itemList = self.sortItems(criterionItem, dummyDemand)                  
        periodList = self.sortPeriods(criterionPeriod, dummyDemand)           
        dl = DemandList(self.nonzerodemand)
        cdef int pos = 0
        cdef int i, t
        if choice == "PI":
            for t in periodList:
                for i in itemList:
                    if dummyDemand[i][t] > MyAbsTol:
                        dl.c_item_index[pos] = i
                        dl.c_period_index[pos] = t
                        dl.c_amounts[pos] = dummyDemand[i][t]
                        pos += 1
        elif choice == "IP":
            for i in itemList:
                for t in periodList:
                    if dummyDemand[i][t] > MyAbsTol:
                        dl.c_item_index[pos] = i
                        dl.c_period_index[pos] = t
                        dl.c_amounts[pos] = dummyDemand[i][t]
                        pos += 1
        else:
            print("Invalid choice!")
        return dl
    
    # generate demand list with specific sorting critieria (only for capactiy-based criteria)
    # returns demand list
    def generateDLdemandOnline(self, str criterionPeriod, str criterionDemand, dummyDemand='Not provided'):
        if dummyDemand == 'Not provided':
            dummyDemand = self.c_demand
        dl = DemandList(self.nonzerodemand)
        periodList = self.sortPeriods(criterionPeriod, dummyDemand)
        cdef int pos = 0
        cdef int i
        for p in periodList:
            itemlist = []
            for it in range(0, self.items):
                itemlist.append(it)
            if criterionDemand == "D":
                itemlist.sort(key=lambda x: dummyDemand[x, p], reverse=True)
            elif criterionDemand == "DH":
                itemlist.sort(key=lambda x: dummyDemand[x, p] / self.holdingCost[x])
            elif criterionDemand == "DS":
                itemlist.sort(key=lambda x: dummyDemand[x, p] / self.setupCost[x])
            elif criterionDemand == "DHS":
                itemlist.sort(key=lambda x: dummyDemand[x, p] * self.holdingCost[x] / self.setupCost[x])
            elif criterionDemand == "DSH":
                itemlist.sort(key=lambda x: dummyDemand[x, p] / (self.holdingCost[x] * self.setupCost[x]))
            else:
                print("Invalid Criterion!")
            for i in itemlist:
                if dummyDemand[i][p] > MyAbsTol:
                    dl.c_item_index[pos] = i
                    dl.c_period_index[pos] = p
                    dl.c_amounts[pos] = dummyDemand[i][p]
                    pos += 1
        return dl
    
    # generates demand list to simulate scenario
    # returns demand list
    def generateOnline(self, str mode='random'):
        
        cdef int pos = 0
        cdef int i
        cdef int p
        cdef int o_start = 0
        
        dl = DemandList(self.nonzerodemand)
        
        # Demand becomes known completely random
        if mode == "random":
            order = list(np.arange(self.items,self.items*self.periods))
            order = random.sample(order, len(order))
            for o in order:
                p = int(o//self.items)
                i = int(o%self.items)
                if self.c_demand[i][p] > MyAbsTol:
                    dl.c_item_index[pos] = i
                    dl.c_period_index[pos] = p
                    dl.c_amounts[pos] = self.c_demand[i][p]
                    pos += 1
        
        # Demand is sorted by increasing periods but random withion the period
        elif mode == "realistic":              
            periodList = self.sortPeriods('INCP')
            itemList = self.sortItems('RAND')  
            for t in periodList:
                for i in itemList:
                    if self.c_demand[i][t] > MyAbsTol:
                        dl.c_item_index[pos] = i
                        dl.c_period_index[pos] = t
                        dl.c_amounts[pos] = self.c_demand[i][t]
                        pos += 1
        
        else:
            print('Invalid mode!')
        return dl

    # sort demand list with incomplete demand matrix
    # returns demand list
    def accumulativeSort(self, DemandList dummyDL, str mode = 'accumulative_cost-based', str criterionPeriod='INCP',
                   str criterionDemand='D', str criterionItem='EC', str choice='PI'):
        
        dummyDemand = np.zeros((self.items, self.periods), dtype=float, order="C")
        for pos in range(dummyDL.length):
            if dummyDL.c_amounts[pos] > MyAbsTol:
                dummyDemand[dummyDL.c_item_index[pos]][dummyDL.c_period_index[pos]] = self.demand[dummyDL.c_item_index[pos]][dummyDL.c_period_index[pos]]
        if mode == 'accumulative_cost-based':
            dl = self.generateDLdemandOnline(criterionPeriod, criterionDemand, dummyDemand)
        elif mode == "accumulative_capacity-based":
            dl = self.generateDLOnline(criterionPeriod, criterionItem, choice, dummyDemand)
        else:
            print('Invalid mode!')
        return dl    
 
    # plan a demand list from pos1 till pos2
    # returns total cost of production plan
    cpdef double planDemandList(self, DemandList dl, uint8 resetSol = True, int pos1 = 0, int pos2 = 1000000000, uint8 resetPostDL = True):
        pos2 = min(pos2, self.nonzerodemand)
        if resetSol:                # reset production plan if true, otherwise continue with plan
            self.resetSolution()
        if resetPostDL:
            self.reset_postponementDL()
        
        cdef int pos
        for pos in range(pos1,pos2): # go through main demand list
            self.usePostDL = 1
            self.deactivatePostponement = False
            self.addDemand(dl.c_item_index[pos], dl.c_period_index[pos], dl.c_amounts[pos]) # add demand element to the partial production plan
            for pos_in_DL2 in range(0, self.nextPos): # go through postponement demand list
                self.keepInPostDL = False
                if self.usePostDL and (dl.c_item_index[pos] == self.postponementDL.c_item_index[pos_in_DL2]):
                    self.addDemand(self.postponementDL.c_item_index[pos_in_DL2], self.postponementDL.c_period_index[pos_in_DL2], self.postponementDL.c_amounts[pos_in_DL2], True, False)
                    if not self.keepInPostDL:
                        self.postponementDL.c_item_index[pos_in_DL2] = -1 # "delete" this element
                        
        # after we went through the whole main demand list, ensure that nothing remained unscheduled in the postponement demand list
        self.DL = 1 
        for pos in range(self.nextPos):
            if self.postponementDL.c_item_index[pos] == -1:
                pass
            else:
                self.addDemand(self.postponementDL.c_item_index[pos], self.postponementDL.c_period_index[pos], self.postponementDL.c_amounts[pos], True, False)
        return self.totalCost

# Run 2-SCH for ABC instances (online environment)
In the following cells, we run 2-SCH on 20 instances from the ABC set. 

For reproducibility purposes, we always set seed to random numbers generator.

The result is saved in a Pandas dataframe.

- [Random scenario](#random)
- [Realistic scenario](#realistic)
- [Accumulative scenario](#accumulative)

<a id="random"></a>

### Random scenario
Postponement is deactivated.

In [6]:
df_random = pd.DataFrame()
row = 0

for seed in range(123, 323): # 200 runs
    random.seed(seed)
    for filename in file_names_abc[:20]:
#         print(filename)
        sol = CLSP(instance_path_abc, filename, setuptimezero = True, storage = 1,
                        shiftProduction=True, postponement = False, threshold=0.5)
        dl = sol.generateOnline(mode='random') # set mode='random' for random scenario
        obj = sol.planDemandList(dl)
        df_random.loc[row, 'Seed'] = seed
        df_random.loc[row, 'instance'] = filename
        df_random.loc[row, 'OBJ'] = obj
        df_random.loc[row, 'HC'] = sol.totalHoldingCost
        df_random.loc[row, 'SC'] = sol.totalSetupCost
        df_random.loc[row, 'OC'] = sol.totalOvertimeCost
        row += 1             

In [7]:
df_random.head(10)

,Seed,instance,OBJ,HC,SC,OC
0,123.0,ABC_1.txt,1.717700e+04,4491.000000,12686.0,0.0
1,123.0,ABC_10.txt,1.209800e+04,2077.000000,10021.0,0.0
2,123.0,ABC_100.txt,4.320100e+04,14575.000000,28626.0,0.0
3,123.0,ABC_101.txt,1.103325e+07,16109.833333,37138.0,10980000.0
4,123.0,ABC_102.txt,3.163550e+06,8634.416667,24916.0,3130000.0
5,123.0,ABC_103.txt,5.143156e+06,15935.266667,47221.0,5080000.0
6,123.0,ABC_104.txt,4.609080e+05,13798.000000,37110.0,410000.0
7,123.0,ABC_105.txt,1.202768e+07,16265.300000,51414.0,11960000.0
8,123.0,ABC_106.txt,4.872707e+06,10871.066667,31836.0,4830000.0
9,123.0,ABC_107.txt,8.449813e+06,9862.250000,29951.0,8410000.0


<a id="realistic"></a>

### Realistic scenario
Demand for earlier periods becomes known earlier: demand list is sorted by increasing periods but random order within a period.

In [8]:
df_realistic = pd.DataFrame()
row = 0

for seed in range(123, 323): # 200 runs
    random.seed(seed)
    for filename in file_names_abc[:20]:
#         print(filename)
        sol = CLSP(instance_path_abc, filename, setuptimezero = True, storage = 1,
                        shiftProduction=True, postponement = True, threshold=0.5)
        dl = sol.generateOnline(mode='realistic') # set mode='realistic' for realistic scenario
        obj = sol.planDemandList(dl)
        df_realistic.loc[row, 'Seed'] = seed
        df_realistic.loc[row, 'instance'] = filename
        df_realistic.loc[row, 'OBJ'] = obj
        df_realistic.loc[row, 'HC'] = sol.totalHoldingCost
        df_realistic.loc[row, 'SC'] = sol.totalSetupCost
        df_realistic.loc[row, 'OC'] = sol.totalOvertimeCost
        row += 1             

In [9]:
df_realistic.head(10)

,Seed,instance,OBJ,HC,SC,OC
0,123.0,ABC_1.txt,16597.000000,5461.000000,11136.0,0.0
1,123.0,ABC_10.txt,11983.000000,2596.000000,9387.0,0.0
2,123.0,ABC_100.txt,43301.000000,14548.000000,28753.0,0.0
3,123.0,ABC_101.txt,55056.666667,12992.666667,42064.0,0.0
4,123.0,ABC_102.txt,36649.850000,8030.850000,28619.0,0.0
5,123.0,ABC_103.txt,64742.450000,13959.450000,50783.0,0.0
6,123.0,ABC_104.txt,52625.250000,12480.250000,40145.0,0.0
7,123.0,ABC_105.txt,69281.750000,13313.750000,55968.0,0.0
8,123.0,ABC_106.txt,45140.200000,7635.200000,37505.0,0.0
9,123.0,ABC_107.txt,40822.083333,7140.083333,33682.0,0.0


<a id="accumulative"></a>

### Accumulative scenario
Demand becomes known randomly but it is allowed to wait before planning. Thus, the accumulated demand elements can be sorted.

In the following implementation, we generate completely random demand list first to later iteratively apply 2-SCH with the reduced set of sorting rules to every 10 demand elements. The partial plan with the minimal cost is selected at the end of each iteration. The number of accumulated elements is controled by parameter variable `n`. 

In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

df_acc = pd.DataFrame()
row = 0

for seed in range(123, 323): # 200 runs
    random.seed(seed)
    for filename in file_names_abc[:20]:
#         print(filename)
        
        sol = CLSP(instance_path_abc, filename, setuptimezero = True, storage = 2, # storage=2 -> we need an additional container to store the best partial plan of the current iteration
                        shiftProduction=True, postponement = True, threshold=0.5)
        
        dl = sol.generateOnline(mode='random')         # generate random demand list
        
        sol.storeSolution(1)                           # store initial solution
        
        n = 10                                         # number of accumulated demand elements
        
        for it in range(0, sol.nonzerodemand, n):
            dummyDL = DemandList(sol.nonzerodemand)
            dummyDL = dl.copy(it, it+n)                # dummyDL holds now the accumulated demand for current iteration only
            best_obj = 1e+12                           # best cost of partial plan for this iteration
            
            # apply cost-based sorting rules
            for pc in period_crit:
                    for ic in cost_crit:
                        for k in key_crit:
                            rule = pc+'-'+ic+'-'+k
                            if rule in reduced_set:
                                tryDL = sol.accumulativeSort(dummyDL, 'accumulative_cost-based', 
                                                           criterionPeriod=pc, criterionItem=ic, choice=k)
                                sol.retrieveSolution(1)
                                obj = sol.planDemandList(tryDL, resetSol = False)
                                if obj < best_obj:
                                    sol.storeSolution(2) # if the better plan is found, store it in additional container
                                    best_obj = obj
            
            # apply capacity-based sorting rules
            for pc in period_crit:
                for dc in capacity_crit:
                    rule = pc+'-'+dc
                    if rule in reduced_set:
                        tryDL = sol.accumulativeSort(dummyDL, 'accumulative_capacity-based', 
                                                   criterionPeriod=pc, criterionDemand=dc)
                        sol.retrieveSolution(1)
                        obj = sol.planDemandList(tryDL, resetSol = False)
                        if obj < best_obj:
                            sol.storeSolution(2)        # if the better plan is found, store it in additional container
                            best_obj = obj
            
            # use the best found solution so far as start for the next iteration
            sol.retrieveSolution(2)
            sol.storeSolution(1)
            
        df_acc.loc[row, 'Seed'] = seed
        df_acc.loc[row, 'instance'] = filename
        df_acc.loc[row, 'OBJ'] = obj
        df_acc.loc[row, 'HC'] = sol.totalHoldingCost
        df_acc.loc[row, 'SC'] = sol.totalSetupCost
        df_acc.loc[row, 'OC'] = sol.totalOvertimeCost
        row += 1             

In [11]:
df_acc.head(10)

,Seed,instance,OBJ,HC,SC,OC
0,123.0,ABC_1.txt,1.705500e+04,4708.000000,12347.0,0.0
1,123.0,ABC_10.txt,1.208200e+04,2213.000000,9869.0,0.0
2,123.0,ABC_100.txt,4.301300e+04,13697.000000,29316.0,0.0
3,123.0,ABC_101.txt,3.133479e+06,15646.333333,37833.0,3080000.0
4,123.0,ABC_102.txt,3.573227e+06,10175.333333,23052.0,3540000.0
5,123.0,ABC_103.txt,3.985967e+06,16288.100000,49679.0,3920000.0
6,123.0,ABC_104.txt,1.021138e+06,13385.000000,37574.0,970000.0
7,123.0,ABC_105.txt,1.290404e+07,13872.250000,50172.0,12840000.0
8,123.0,ABC_106.txt,3.822721e+06,10623.600000,32097.0,3780000.0
9,123.0,ABC_107.txt,8.448482e+06,9671.083333,28811.0,8410000.0
